In [ ]:
!pip install shap
!pip install pdpbox

     |███████████████████████████████▊| 368 kB 4.0 MB/s eta 0:00:01

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/bootcamp/project2/phishing/phishing dataset.zip')
df.shape

In [ ]:
df.info()

# Columns
id                                    <br>
NumDots                               
 .의 갯수<br> 
SubdomainLevel                        
 서브도메인 레벨<br> 
PathLevel                             
 패치의레벨<br> 
UrlLength                             
 주소길이<br> 
NumDash                               
 -갯수<br> 
NumDashInHostname                     
host name 안에 - 의 갯수<br> 
AtSymbol                              
@갯수<br> 
TildeSymbol                           
~의 갯수<br> 
NumUnderscore                         
_갯수<br> 
NumPercent                            
%갯수<br> 
NumQueryComponents                    
? 바로 뒤에 오는 component<br> 
NumAmpersand                          
&의 갯수<br> 
NumHash  
'#'의 갯수 <br> 
NumNumericChars                       
숫자갯수<br> 
NoHttps                               
https를 사용하지 않음<br> 
RandomString                          
아무문자열<br> 
IpAddress                             
ip주소<br> 
DomainInSubdomains                    
서브도메인의 도메인<br> 
DomainInPaths                         
path의 도메인<br> 
HttpsInHostname                       
hostname안에 https포함<br> 
HostnameLength                        
hostname길이<br> 
PathLength                            
path길이<br> 
QueryLength                           
쿼리 길이<br> 
DoubleSlashInPath                     
//가 path안에있는지<br> 
NumSensitiveWords                     
민감한단어의 갯수<br> 
EmbeddedBrandName                     
브랜드 이름의 포함<br> 
PctExtHyperlinks                      
외부하이퍼링크?<br> 
PctExtResourceUrls                    
외부 리소스url<br> 
ExtFavicon                            
외부 즐겨찾기?<br> 
InsecureForms                         
안전하지 않은 양식<br> 
RelativeFormAction                    
반응형 사이트(상대 형식 동작)<br> 
ExtFormAction                         
외부 액션폼<br> 
AbnormalFormAction                    
비이상적인 양식 작업<br>
PctNullSelfRedirectHyperlinks         
Null 자체 리디렉션 하이퍼링크<br>
FrequentDomainNameMismatch            
빈번한 도메인 이름이 잘못 일치함<br>
FakeLinkInStatusBar                   
가짜 링크 상태 표시줄<br>
RightClickDisabled                    
마우스 오른쪽 버튼 사용 안 함<br>
PopUpWindow                           
팝업 창<br>
SubmitInfoToEmail                     
전자 메일에 정보 제출<br>
IframeOrFrame                         
Iframe 또는 프레임 <br>
MissingTitle                          
미싱타이틀(타이틀이 없음)<br>
ImagesOnlyInForm                      
형식만 있는 이미지<br>
SubdomainLevelRT                      
하위 도메인 레벨 RT(Route)<br>
UrlLengthRT                           
url길이(route)<br>
PctExtResourceUrlsRT                  
외부리소스(RT)<br>
AbnormalExtFormActionRT               
비정상적인 확장 양식 작업(route)<br>
ExtMetaScriptLinkRT                   
외부 메타 스크립트 링크(route)<br>
PctExtNullSelfRedirectHyperlinksRT    
외부NUll 자체 리디렉션 하이퍼링크 (route)<br>
CLASS_LABEL                           
라벨<br>

In [ ]:
df.isna().sum()

In [ ]:
df.shape

메모리 절약을 위해서 64bit 에서 32bit로 바꾸어준다

In [ ]:
def col_type_change(col,col_type):
    for i in col:
        df[i] = df[i].astype(col_type)
    return df
    
float_columns = df.select_dtypes('float64').columns
int_columns = df.select_dtypes('int64').columns


df = col_type_change(float_columns,"float32")
df = col_type_change(int_columns,"int32")

# df.info()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.CLASS_LABEL.value_counts()

1    5000
0    5000
Name: CLASS_LABEL, dtype: int64

In [ ]:
df.drop('id',axis=1,inplace=True)
print(df.columns)
target = 'CLASS_LABEL'
features = df.drop([target],axis=1).columns

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

train,test = train_test_split(df,train_size=0.80, test_size=0.20, random_state=2)
train,val = train_test_split(train,train_size=0.80, test_size=0.20, random_state=2)

train.shape,val.shape

In [ ]:
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]
y_test = test[target]

print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

# RandomForest Model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score


model = RandomForestClassifier(random_state=2)
model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print(classification_report(y_pred, y_val))

fig, ax = plt.subplots()
pcm = plot_confusion_matrix(model, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax);
plt.title(f'Confusion matrix, n = {len(y_val)}', fontsize=15);
auc_score = roc_auc_score(y_val, y_pred)
print("AUC_SCORE:",auc_score)
print()

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
n = 20
importances = pd.Series(model.feature_importances_, X_val.columns)
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh();

##기준모델 

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import classification_report


model = XGBClassifier(random_state=2)

model.fit(X_train,y_train)
y_pred = model.predict(X_val)
print(classification_report(y_pred, y_val))

fig, ax = plt.subplots()
pcm = plot_confusion_matrix(model, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax);
plt.title(f'Confusion matrix, n = {len(y_val)}', fontsize=15);

auc_score = roc_auc_score(y_val, y_pred)
print("AUC_SCORE:",auc_score)

##모델학습

In [ ]:
model2 = XGBClassifier(n_estimators=200
                  , random_state=2
                  , n_jobs=-1
                  , max_depth=7
                  , learning_rate=0.2)

model2.fit(X_train,y_train)
y_pred = model.predict(X_val)
print(classification_report(y_pred, y_val))

fig, ax = plt.subplots()
pcm = plot_confusion_matrix(model2, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax);
plt.title(f'Confusion matrix, n = {len(y_val)}', fontsize=15);

auc_score = roc_auc_score(y_val, y_pred)
print("AUC_SCORE:",auc_score)

In [ ]:
eval_set = [(X_train, y_train), 
            (X_val, y_val)]

model2.fit(X_train, y_train, 
          eval_set=eval_set,
          eval_metric='error',
          early_stopping_rounds=50
         ) 

In [ ]:
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt

y_pred = model.predict(X_val)
print(classification_report(y_pred, y_val))

fig, ax = plt.subplots()
pcm = plot_confusion_matrix(model, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax);
plt.title(f'Baseline Model matrix, n = {len(y_val)}', fontsize=15);
auc_score = roc_auc_score(y_val, y_pred)
print("AUC_SCORE:",auc_score)

In [ ]:
y_pred = model2.predict(X_val)
print(classification_report(y_pred, y_val))

fig, ax = plt.subplots()
pcm = plot_confusion_matrix(model2, X_val, y_val,
                            cmap=plt.cm.Blues,
                            ax=ax);
plt.title(f'Model2 matrix, n = {len(y_val)}', fontsize=15);
auc_score = roc_auc_score(y_val, y_pred)
print("AUC_SCORE:",auc_score)

일반사이트를 피싱사이트라고 지정하게 되면 문제가 발생할수도 있기 때문에 값을 좀더 조절해본다

In [ ]:
from pprint import pprint

model_ =  XGBClassifier(n_estimators=200
                  , random_state=2
                  , n_jobs=-1
                  , max_depth=7
                  , learning_rate=0.2)

params={
    'max_depth':range(3,10,3),
    'min_child_weight':range(1,6,2),
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
    'subsample':[i/100.0 for i in range(40,80)],
    'colsample_bytree':[i/10.0 for i in range(6,10)],
    'gamma':[i/10.0 for i in range(0,5)]
}


randomsearch = RandomizedSearchCV(model_,
                               param_distributions = params,
                               n_iter = 50,
                               cv = 3,
                               verbose=2, 
                               random_state=2, 
                               n_jobs = -1, 
                               scoring='roc_auc')

randomsearch.fit(X_train, y_train, 
          eval_set=eval_set,
          eval_metric='error',
          early_stopping_rounds=50
         ) 

print(randomsearch.best_score_)
pprint(randomsearch.best_params_)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   55.2s


[0]	validation_0-error:0.030156	validation_1-error:0.043125
Multiple eval metrics have been passed: 'validation_1-error' will be used for early stopping.

Will train until validation_1-error hasn't improved in 50 rounds.
[1]	validation_0-error:0.032969	validation_1-error:0.039375


[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  3.3min finished


[2]	validation_0-error:0.021562	validation_1-error:0.036875
[3]	validation_0-error:0.02375	validation_1-error:0.036875
[4]	validation_0-error:0.022656	validation_1-error:0.039375
[5]	validation_0-error:0.020781	validation_1-error:0.035
[6]	validation_0-error:0.017656	validation_1-error:0.03
[7]	validation_0-error:0.016562	validation_1-error:0.02875
[8]	validation_0-error:0.015625	validation_1-error:0.0275
[9]	validation_0-error:0.01375	validation_1-error:0.02875
[10]	validation_0-error:0.013125	validation_1-error:0.028125
[11]	validation_0-error:0.012656	validation_1-error:0.02625
[12]	validation_0-error:0.0125	validation_1-error:0.024375
[13]	validation_0-error:0.011719	validation_1-error:0.025
[14]	validation_0-error:0.010625	validation_1-error:0.02375
[15]	validation_0-error:0.010625	validation_1-error:0.0225
[16]	validation_0-error:0.009844	validation_1-error:0.021875
[17]	validation_0-error:0.009375	validation_1-error:0.021875
[18]	validation_0-error:0.00875	validation_1-error:0.0

In [ ]:
%%time 

params = {'colsample_bytree': [0.6,0.65],
          'gamma':[i/10.0 for i in range(0,2)],
          'max_depth': [8,9,10],
          'min_child_weight': [1,1.5],
          'reg_alpha': [0.1,0.15],
          'subsample': [0.65,0.7,0.75]}
          
model_ =  XGBClassifier(n_estimators=200
                  , random_state=2
                  , n_jobs=-1
                  , max_depth=7
                  , learning_rate=0.2)


gridsearch = GridSearchCV(model_,
                          param_grid=params,
                          cv = 3,
                          n_jobs = -1, 
                          scoring='roc_auc')

gridsearch.fit(X_train, y_train, 
          eval_set=eval_set,
          eval_metric='error',
          early_stopping_rounds=50
         ) 

print(gridsearch.best_score_)
pprint(gridsearch.best_params_)

y_pred = gridsearch.predict(X_val)



##ROC Curve

In [ ]:
from sklearn.metrics import roc_curve
y_pred = gridsearch.predict_proba(X_val)[:, 1]
fpr, tpr, thresholds = roc_curve(y_val, y_pred)

roc = pd.DataFrame({
    'FPR(Fall-out)': fpr, 
    'TPRate(Recall)': tpr, 
    'Threshold': thresholds
})
plt.scatter(fpr, tpr)
plt.title('ROC curve')
plt.xlabel('FPR(Fall-out)')
plt.ylabel('TPR(Recall)');

optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print('idx:', optimal_idx, ', threshold:', optimal_threshold)

In [ ]:
auc_score = roc_auc_score(y_val, y_pred)
print("AUC_SCORE:",auc_score)

In [ ]:
y_pred =gridsearch.predict_proba(X_test)[:, 1]
auc_score = roc_auc_score(y_test, y_pred)
print("TEST AUC_SCORE:",auc_score)

In [ ]:
import shap

model_final = gridsearch.best_estimator_
explainer = shap.TreeExplainer(model_final)

shap_values = explainer.shap_values(X_test)


shap.initjs()
shap.force_plot(
    base_value=explainer.expected_value, 
    shap_values=shap_values,
    # matplotlib=True,
    features=X_test
)

In [ ]:

shap.summary_plot(shap_values, X_test.iloc[:10], plot_type="bar")

shap_values = explainer.shap_values(X_test.iloc[:300])
shap.summary_plot(shap_values, X_test.iloc[:300])
shap.summary_plot(shap_values, X_test.iloc[:300], plot_type="violin")

In [ ]:

y_pred =gridsearch.predict_proba(X_test)[:, 1]
fig, ax = plt.subplots()
pcm = plot_confusion_matrix(model_final, X_test, y_test,
                            cmap=plt.cm.Blues,
                            ax=ax);
plt.title(f'Model2 matrix, n = {len(y_val)}', fontsize=15);
auc_score = roc_auc_score(y_test, y_pred)
print("AUC_SCORE:",auc_score)

NameError: ignored